In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.17.1


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

test_images = test_images[:1000].reshape(-1,28*28)/255.0
train_images = train_images[:1000].reshape(-1,28*28)/255.0

In [5]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

model = create_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
#użycie callbacks w punktach kontrolnych
checkpoint_path = "training_1/cp.ckpt.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/10
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4654 - loss: 1.7087
Epoch 1: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.4957 - loss: 1.6322 - val_accuracy: 0.7830 - val_loss: 0.7214
Epoch 2/10
31/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8897 - loss: 0.4467
Epoch 2: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8897 - loss: 0.4450 - val_accuracy: 0.8240 - val_loss: 0.5508
Epoch 3/10
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9074 - loss: 0.2871
Epoch 3: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9081 - loss: 0.2865 - val_accuracy: 0.8430 - val_loss: 0.4979
Epoch 4/10
25/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9331 - loss: 0.2485
Epoch 4: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9357 - loss: 0.2413 - val_accuracy: 0.8520

In [7]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 - 0s - 9ms/step - accuracy: 0.0900 - loss: 2.3601
Untrained model, accuracy:  9.00%


In [8]:
model.load_weights(checkpoint_path)

loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))


32/32 - 0s - 4ms/step - accuracy: 0.8700 - loss: 0.4070


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Untrained model, accuracy: 87.00%


In [9]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                 save_freq=5*batch_size)

In [10]:
import os
os.makedirs(checkpoint_dir, exist_ok=True)
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=50,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 5: saving model to training_2/cp-0005.ckpt.weights.h5

Epoch 10: saving model to training_2/cp-0010.ckpt.weights.h5

Epoch 15: saving model to training_2/cp-0015.ckpt.weights.h5

Epoch 20: saving model to training_2/cp-0020.ckpt.weights.h5

Epoch 25: saving model to training_2/cp-0025.ckpt.weights.h5

Epoch 30: saving model to training_2/cp-0030.ckpt.weights.h5

Epoch 35: saving model to training_2/cp-0035.ckpt.weights.h5

Epoch 40: saving model to training_2/cp-0040.ckpt.weights.h5

Epoch 45: saving model to training_2/cp-0045.ckpt.weights.h5

Epoch 50: saving model to training_2/cp-0050.ckpt.weights.h5


In [11]:
os.listdir(checkpoint_dir)

['cp-0045.ckpt.weights.h5',
 'cp-0010.ckpt.weights.h5',
 'cp-0030.ckpt.weights.h5',
 'cp-0035.ckpt.weights.h5',
 'cp-0000.ckpt.weights.h5',
 'cp-0005.ckpt.weights.h5',
 'cp-0025.ckpt.weights.h5',
 'cp-0015.ckpt.weights.h5',
 'cp-0020.ckpt.weights.h5',
 'cp-0040.ckpt.weights.h5',
 'cp-0050.ckpt.weights.h5']

In [14]:
model = create_model()
model.load_weights('training_2/cp-0050.ckpt.weights.h5')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("0050 model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 15ms/step - accuracy: 0.8790 - loss: 0.4773
0050 model, accuracy: 87.90%


In [15]:
model.load_weights('training_2/cp-0015.ckpt.weights.h5')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("0015 model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 5ms/step - accuracy: 0.8780 - loss: 0.4050
0015 model, accuracy: 87.80%


In [16]:
model.load_weights('training_2/cp-0005.ckpt.weights.h5')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("0005 model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 5ms/step - accuracy: 0.8690 - loss: 0.4102
0005 model, accuracy: 86.90%


In [17]:
#zapis całego modelu
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5')

Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5248 - loss: 1.6165
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8443 - loss: 0.4753
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9228 - loss: 0.2908
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9402 - loss: 0.2329
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9625 - loss: 0.1563


In [18]:
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [19]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))
#

32/32 - 0s - 10ms/step - accuracy: 0.8620 - loss: 0.4255
Restored model, accuracy: 86.20%
